In [1]:
import plasma.search_engines as ses
import pandas as pd


df = pd.read_feather('/home/thanh/development/u02/thanhhm/xproject/as/databases/icd10.feather')
engine = ses.GraphMatcher(
    df['name'], 
    group_splitter='([^:\n;?!.()]+)',
    top_k=5,
    token_threshold=0.6
)

/u03/thanhhm/envs/research/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
query = 'Tổn thương nông tác động nhiều vùng cơ thể, Đa chấn thương, vết thương cẳng chân trái do tai nạn giao thông'
results = engine.run(query)
results

{'GraphMatcher._compare_tokens': '0.61s'}
{'PathWalker._walk_path': '0.00s'}
{'GraphMatcher._analyze_path': '0.15s'}
{'GraphMatcher._map_data': '1.13s'}
{'GraphMatcher.run': '2.81s'}


data_index  \
query_start_index query_end_index               
0                 15                    19628   
                  15                    19639   
                  15                    19641   
                  15                    19827   
                  15                    19632   
11                19                    26033   
16                42                    21271   
                  42                    21359   
                  42                    21867   
                  42                    21378   
                  42                    21879   
44                58                    21377   
60                70                     2218   
                  70                    19650   
                  70                    19657   
                  70                    19659   
                  70                    19840   
64                80                    21018   
                  80                    20988   
                  80                    21025   
                  80                    21019   
                  80                    21020   
76                85                     4548   
86                107                   24359   
                  107                   24379   
                  107                   24358   
                  107                   24378   
                  107                   24361   

                                                                       original_text  \
query_start_index query_end_index                                                      
0                 15                                           Tổn thương nông ở đầu   
                  15                                         Tổn thương nông của mũi   
                  15                                         Tổn thương nông của tai   
                  15                                          Tổn thương nông của cổ   
                  15                                      Tổn thương nông của da đầu   
11                19                                    Khám trước khi nhận công tác   
16                42                           Tổn thương tác động nhiều vùng cơ thể   
                  42                  Chấn thương đứt lìa tác động nhiều vùng cơ thể   
                  42               Tổn thương do cóng lạnh tác động nhiều vùng cơ...   
                  42               Tổn thương khác tác động nhiều vùng cơ thể, kh...   
                  42               Tổn thương do cóng lạnh không xác định tác độn...   
44                58                           Đa chấn thương đứt lìa không xác định   
60                70                                       Bệnh giòi trên vết thương   
                  70                                             Vết thương hở ở đầu   
                  70                                           Vết thương hở của mũi   
                  70                                           Vết thương hở của tai   
                  70                                            Vết thương hở của cổ   
64                80                                Gãy nhiều vị trí xương cẳng chân   
                  80                            Gãy xương cẳng chân, bao gồm cổ chân   
                  80                        Gãy xương cẳng chân, phần không xác định   
                  80                       Gãy nhiều vị trí xương cẳng chân, gãy kín   
                  80                        Gãy nhiều vị trí xương cẳng chân, gãy hở   
76                85                                     Hội chứng tái tạo miễn dịch   
86                107              Người đi xe nông nghiệp bị thương không phải d...   
                  107              Người đi xe xây dựng bị thương không phải do t...   
                  107              Lái xe của xe nông nghiệp bị thương không phải...   
                  107              Lái xe của xe xây dựng bị thương không phải do...   
            

In [ ]:
import networkx as nx
import numpy as np
import pandas as pd

In [ ]:
df = pd.read_pickle('debug.pkl')

In [ ]:
sequential_db_tokens = df['db_tokens']

In [ ]:
def construct_structure(position_index, token, cached_structure):
    cached_structure[position_index, token] = []
    if position_index < len(sequential_db_tokens) - 1:
        next_index = position_index + 1
        for next_token in sequential_db_tokens.iloc[next_index].index:
            if (token, next_token) in engine._graph.edges:
                cached_structure[position_index, token].append((next_index, next_token))

paths = {}
for i in range(len(sequential_db_tokens)):
    for tk in sequential_db_tokens.iloc[i].index:
        construct_structure(i, tk, paths)

len(paths)

In [ ]:
def render_path(position_index, token, cached_paths):
    next_paths = paths[position_index, token]
    token_score = sequential_db_tokens.iloc[position_index].loc[token]
    if len(next_paths) == 0:
        cached_paths[position_index, token] = [{
            'path': (token,),
            'score': token_score,
            'len': 1,
        }]
    else:
        cached_paths[position_index, token] = []
        for next_position, next_token in next_paths:
            if (next_position, next_token) not in cached_paths:
                render_path(next_position, next_token, cached_paths)

            rendered_paths = cached_paths[next_position, next_token]
            for p in rendered_paths:
                new_len = p['len'] + 1
                new_score = new_len / (1 / token_score + p['len'] / p['score'])
                if new_score >= engine.path_walker.threshold:
                    cached_paths[position_index, token].append({
                        'path': (token, p), 
                        'score': new_score,
                        'len': new_len
                    })

cached = {}
for k in paths:
    render_path(*k, cached)

len(cached)

In [ ]:
for k, v in cached.items():
    print(k, len(v))

In [ ]:
cached

In [ ]:
def flatten(path):
    if len(path) == 1:
        return path[0],
    else:
        return (path[0], *flatten(path[1]))

In [ ]:
flattens = []
for (position_index, _), temp_paths in cached.items():
    for p in temp_paths:
        flattens.append((position_index, flatten(p)))


In [ ]:
import scipy.stats as stats

In [ ]:
final_paths = []
for i, p in flattens:
    scores = [df.iloc[i + j]['db_tokens'].loc[tk] for j, tk in enumerate(p)]
    if stats.hmean(scores) >= engine.path_walker.threshold:
        final_paths.append(p)

In [ ]:
i, p

In [ ]:
paths